In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt

from micro_price_trading import Preprocess, TwoAssetSimulation
from micro_price_trading.config import TWENTY_SECOND_DAY, DATA_PATH

In [2]:
raw = Preprocess('Test_TBT_TBF_data.csv')
data = raw.process()

In [3]:
with open('simulations.npy', 'rb') as file:
    input_data = np.load(file)

In [4]:
q_values = pd.read_csv(DATA_PATH.joinpath('q_values_9_27.csv'), index_col=0)

In [5]:
def CI(array, confidence = .95):
    half_width = 1.96*array.std()/np.sqrt(len(array))
    mu = array.mean()
    return mu - half_width, mu + half_width

In [6]:
def optimal_twap(t):
    indices_to_buy_at = [idx for idx in range(0, TWENTY_SECOND_DAY, t)]
    q_values_binary = (q_values.iloc[input_data[0,indices_to_buy_at,0]]['Long/Short'] >
                       q_values.iloc[input_data[0,indices_to_buy_at,0]]['Short/Long']).values
    
    indices_to_buy_asset_1 = np.where(q_values_binary)[0]
    indices_to_buy_asset_2 = np.where(~q_values_binary)[0]
    
    avg_prices_1 = input_data[:,indices_to_buy_asset_1, 1].mean(axis = 1)
    avg_prices_2 = input_data[:,indices_to_buy_asset_2, 2].mean(axis = 1)

    return avg_prices_1, avg_prices_2

In [ ]:
(CI(optimal_twap(10)[0]),
 CI(optimal_twap(10)[1]))

((18.29869510546961, 18.301664288469784),
 (16.65529484796928, 16.65670668144248))